In [14]:
import os
import cv2
import numpy as np
import albumentations as A

# Set the paths for input and output directories
input_dir = "CVC-ClinicDB/Preprocessed"
mask_dir = "CVC-ClinicDB/Preprocessed_mask"
output_dir = "CVC-ClinicDB/Augmented"
output_mask_dir = "CVC-ClinicDB/Augmented_mask"

# Create output directories if they don't exist
os.makedirs(output_dir, exist_ok=True)
os.makedirs(output_mask_dir, exist_ok=True)

# Get the list of image files
image_files = sorted(os.listdir(input_dir))


# Define the augmentation pipeline
augmentation_pipeline = A.Compose([
    A.HorizontalFlip(p=0.3),
    A.VerticalFlip(p=0.3),
    A.RandomRotate90(p=0.3),
    A.Transpose(p=0.3),
    A.OneOf([
        A.RandomContrast(),
        A.RandomGamma(),
        A.RandomBrightness(),
    ], p=0.3),
    A.OneOf([
        A.ElasticTransform(alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
        A.GridDistortion(),
        A.OpticalDistortion(distort_limit=2, shift_limit=0.5),
    ], p=0.3),
    A.OneOf([
        A.Blur(blur_limit=3),
        A.MedianBlur(blur_limit=3),
        A.GaussianBlur(blur_limit=3),
    ], p=0.3),
    A.IAAAdditiveGaussianNoise(p=0.3),  # Add Gaussian noise
    A.RandomCrop(width=128, height=128, p=0.1),  # Add random cropping
    A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=20, p=0.3),  # Add translation
    A.RGBShift(p=0.3),  # Add color space shifting
    A.RandomBrightnessContrast(p=0.3),  # Add brightness adjustment
])

# Perform data augmentation and save the augmented images and masks
for image_file in image_files:
    # Load the image and mask
    image_path = os.path.join(input_dir, image_file)
    mask_path = os.path.join(mask_dir, image_file)

    image = cv2.imread(image_path)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    # Perform augmentation
    for i in range(1, 11):  # You can change the range based on how many augmentations you want
        
        output_image_path = os.path.join(output_dir, f"{base_name}_{i}.tif")
        output_mask_path = os.path.join(output_mask_dir, f"{base_name}_{i}.tif")
        
        if i==1:
            cv2.imwrite(output_image_path, image)
            cv2.imwrite(output_mask_path, mask)
        else:
            augmented = augmentation_pipeline(image=image, mask=mask)
            augmented_image = augmented['image']
            augmented_mask = augmented['mask']
    
            # Save augmented images and masks
            base_name = os.path.splitext(image_file)[0]
            cv2.imwrite(output_image_path, augmented_image)
    
            # Save corresponding augmented mask
            cv2.imwrite(output_mask_path, augmented_mask)

In [15]:
3+2

5